In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re
import seaborn as sns
import time
import random
import datetime
from fund_analysis import *






# Choose Sample Size

NOTE: sample size represents the size for both the active and passive mutual funds (not the combined sample size)

In [2]:
sample_size = 3000

In [3]:
data = pd.read_csv('CRSP.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,12,49,50,51,62,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
start = pd.to_datetime('01/01/1988')
end = pd.to_datetime('04/16/2018')


In [5]:
df = pd.DataFrame({'Fund Name' : data['fund_name'], 'Fund Ticker' : data['ticker'], 
                   'Open to Investors' : data['open_to_inv'], 'Expense Ratio' : data['exp_ratio'],
                  'Management Fee' : data['mgmt_fee'], 'Turnover Ratio' : data['turn_ratio']})

In [6]:
df.dropna(inplace = True)

In [7]:
df['Active or Passive'] = df['Expense Ratio'].apply(managed)
df['Active or Passive'] = df['Turnover Ratio'].apply(managed_turn)

In [8]:
df = df[df['Open to Investors'] == 'Y']

In [9]:
df = df[df['Management Fee'] > 0]

In [10]:
df = df[df['Expense Ratio'] > 0]

In [11]:
df.drop_duplicates(subset = ['Fund Ticker'], inplace = True)

In [12]:
df_active = df[df['Active or Passive'] == 'active']
df_passive = df[df['Active or Passive'] != 'active']

In [13]:
df_active_sample = df_active.sample(sample_size)
df_passive_sample = df_passive.sample(sample_size)

In [ ]:
active_price_data = {}
passive_price_data = {}

In [ ]:
# let's create a dictionary where the key is the ticker symbol and the value is the historical NAV of the mutual fund

counter = 0


print('Progress:')
print('')
for fund in df_active_sample['Fund Ticker']:
    # don't want to get blocked by yahoo
    sleeps = random.randint(4,17)
    time.sleep(sleeps)
    try:
        active_price_data[fund] = get_yahoo_prices(fund, start, end).reset_index()[['date','close']]
        counter += 1
        print(counter)


    except:
        counter += 1
        print(counter)
        pass
        
for fund in df_passive_sample['Fund Ticker']:
    # don't want to get blocked by yahoo
    sleeps = 4
    time.sleep(sleeps)
    try:
        passive_price_data[fund] = get_yahoo_prices(fund, start, end).reset_index()[['date','close']]
        counter += 1
        print(counter)
    except:
        counter += 1
        print(counter)
        pass


In [ ]:
# let's create a dataframe for the active and passive mutual funds where the first column is a date range

active_dated = pd.DataFrame({'date' : pd.date_range('1/1/1980', '4/19/2018')})
passive_dated = pd.DataFrame({'date' : pd.date_range('1/1/1980', '4/19/2018')})

In [ ]:
# let's merge the above dataframe with the data we have (stored in dictionaries)

active_price = merged(active_dated, active_price_data)
passive_price = merged(passive_dated, passive_price_data)

In [ ]:
# let's make the dataframes the same size

if len(active_price.columns) != len(passive_price.columns):
        if len(active_price.columns) > len(passive_price.columns):
            num = len(active_price.columns) - (len(active_price.columns) - len(passive_price.columns))
            active_price = active_price.iloc[:,:num]
        else:
            num = len(passive_price.columns) - (len(passive_price.columns) - len(active_price.columns))
            passive_price = passive_price.iloc[:,:num]

In [ ]:
# stores pricing data to csv

passive_price.to_csv('passive_price_data.csv')
active_price.to_csv('active_price_data.csv')